### Quickstart: {Compare runs, choose a model and reploy it to a REST API}

#### Run hyperaparameter sweep on a training script
#### compare results of runs in the MLflow UI
#### Choose best run and register it as a model
#### deploy model to a REST API
#### Build a container image suitable for deployment to a cloud platform

In [21]:
import keras
import numpy as np
import pandas as pd
import mlflow
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from mlflow.models import infer_signature

In [22]:
data = pd.read_csv("https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",sep=';')

In [23]:
#split the data into training, validation and test sets

train,test=train_test_split(data,test_size=.25,random_state=42)

#training dataset
train_x = train.drop(['quality'],axis=1).values
train_y = train[['quality']].values.ravel() #.ravel() allows us to get a single dimension array, otherwise .values() produces a 2D array

#test dataset
test_x = test.drop(['quality'],axis=1).values
test_y = test[['quality']].values.ravel()

#split the train data into train validation AGAIN, to check the performance of the model
train_x,valid_x,train_y,valid_y=train_test_split(train_x,train_y,test_size=.2,random_state=42)

signature = infer_signature(train_x,train_y)

In [24]:
### ANN Model

def train_model(params,epochs,train_x,train_y,valid_x,valid_y,test_x,test_y):
    ## Define model architecture
    mean = np.mean(train_x,axis=0) #the mean of each feature, for normalization
    var = np.var(train_x,axis=0)

    model=keras.Sequential([
        keras.Input([train_x.shape[1]]), #11 input features in the input layer
        keras.layers.Normalization(mean=mean,variance=var), #apply normalization to input features
        keras.layers.Dense(64,activation='relu'), #hidden layers
        keras.layers.Dense(1) #output layer
    ])

    ## compile the model
    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate=params['lr'],momentum=params['momentum'] #hyper parameters
        ),
        loss='mean_squared_error', #tracking
        metrics=[keras.metrics.RootMeanSquaredError()]
    )

    ## Train the ANN model with lr and momentum params with MLFLOW tracking
    with mlflow.start_run(nested=True):
        model.fit(train_x,train_y,validation_data=(valid_x,valid_y), epochs=epochs, batch_size=64)

        ## Evaluate the model
        eval_result = model.evaluate(valid_x,valid_y,batch_size=64)

        eval_rmse=eval_result[1]

        ## log the parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse",eval_rmse)

        ## log the model
        mlflow.tensorflow.log_model(model, 'model',signature=signature)

        return {'loss':eval_rmse,"status":STATUS_OK,"model":model}

In [25]:
def objective(params):
    # MLflow will track params and results for each run
    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y
    )
    return result

In [26]:
space={
    "lr":hp.loguniform("lr",np.log(1e-5),np.log(1e-1)), #establishes the learning rate range
    'momentum':hp.uniform('momentum',0.0,1.0) #momentum param range
}

In [27]:
mlflow.set_experiment("/wine-quality")
with mlflow.start_run():
    #conduct the hyperparameter search using Hyperopt
    trials = Trials()
    best=fmin(
        fn=objective,
        space=space,
        max_evals=4,
        trials=trials
    )
    
    #fecth the details of the best run
    best_run=sorted(trials.results,key=lambda x: x['loss'])[0]

    #log the best paramters, loss and model
    mlflow.log_params(best)
    mlflow.log_metric(key='eval_rmse', value= best_run['loss'])
    mlflow.tensorflow.log_model(best_run['model'], 'model', signature=signature)

    print(f'Best parameters: {best}')
    print(f'Best eval rmse: {best_run['loss']}')

    mlflow.set_tracking_uri('http://127.0.0.1:5000')



2025/01/09 21:01:28 INFO mlflow.tracking.fluent: Experiment with name '/wine-quality' does not exist. Creating a new experiment.
TPE is being used as the default algorithm.


Epoch 1/3                                            

  0%|          | 0/4 [00:00<?, ?trial/s, best loss=?]

2025-01-09 21:01:28.735724: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


 1/46 ━━━━━━━━━━━━━━━━━━━━ 34s 756ms/step - loss: 42.2024 - root_mean_squared_error: 6.4963
11/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 15.1715 - root_mean_squared_error: 3.7257   
33/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 8.0035 - root_mean_squared_error: 2.6331 
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 6.4048 - root_mean_squared_error: 2.3321 - val_loss: 0.6430 - val_root_mean_squared_error: 0.8019

Epoch 2/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.5452 - root_mean_squared_error: 0.7384
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6086 - root_mean_squared_error: 0.7800 - val_loss: 0.5340 - val_root_mean_squared_error: 0.7308

Epoch 3/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 7s 169ms/step - loss: 0.8692 - root_mean_squared_error: 0.9323
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6209 - root_mean_squared_error: 0.7876 - val_loss: 0.5701 - val_root_mean_squared_error: 0.7551



## Validate Model and Inferencing

In [28]:
from mlflow.models import validate_serving_input

model_uri = 'runs:/71fcdaa1c43b4ca9942c97464f26a2c0/model'

# The logged model does not contain an input_example.
# Manually generate a serving payload to verify your model prior to deployment.
from mlflow.models import convert_input_example_to_serving_input

# Define INPUT_EXAMPLE via assignment with your own input example to the model
# A valid input example is a data instance suitable for pyfunc prediction
serving_payload = convert_input_example_to_serving_input(test_x)

# Validate the serving payload works on the model
validate_serving_input(model_uri, serving_payload)

/home/codespace/.python/current/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

 1/39 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step

39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


array([[6.272072 ],
       [6.7174463],
       [6.5126276],
       ...,
       [5.9886003],
       [6.999872 ],
       [5.755769 ]], dtype=float32)